# Import Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 500)

In [2]:
gg_data = pd.read_csv('googleplaystore.csv')
gg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [3]:
# create a copy of the original
data = gg_data.copy()

#check the value counts of each column and look for any weirdness
for col in data:
    print(col)
    print(data[col].value_counts(normalize=True))
    print('-------------------------------')

App
ROBLOX                                               0.000830
CBS Sports App - Scores, News, Stats & Watch Live    0.000738
Duolingo: Learn Languages Free                       0.000646
ESPN                                                 0.000646
8 Ball Pool                                          0.000646
                                                       ...   
Dosecast - Medication Reminder                       0.000092
BS Films                                             0.000092
BH Recepti                                           0.000092
Brain Waves - Binaural Beats                         0.000092
Forgotten Hill: Fall                                 0.000092
Name: App, Length: 9660, dtype: float64
-------------------------------
Category
FAMILY                 0.181902
GAME                   0.105525
TOOLS                  0.077760
MEDICAL                0.042708
BUSINESS               0.042432
PRODUCTIVITY           0.039111
PERSONALIZATION        0.036159
COMMUNICA

**Observations**
1. **App - there seems to be a few duplicates in app. have to deal with deleting those.**
2. **Category - need to look into '1.9'.**
3. **Rating - the rating is from 1-5, but there's a rating of 19. Need to delete that one.**
4. **Reviews - might drop the '0' reviews or filter the amount of reviews.**
5. **Size - the 'varies with device' need to be replace with the average size and the "M" needs to be taken out.**
6. **Installs - need to get rid of the '+' and maybe filter a min-max scale.**
7. **Change Type into a numeric column**
8. **Change last update into days.**

# Cleaning

## App Duplicates
**Looking at the value counts I noticed that there were a few apps that were duplicates, meaning the name of the app was the same. This column should only have unique names. So I'm going to pull them out and delete all except for the first one.**

In [4]:
#here's an example of an duplicated app
data[data['App'] == 'ROBLOX']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1653,ROBLOX,GAME,4.5,4447388,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up
1701,ROBLOX,GAME,4.5,4447346,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up
1748,ROBLOX,GAME,4.5,4448791,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up
1841,ROBLOX,GAME,4.5,4449882,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up
1870,ROBLOX,GAME,4.5,4449910,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up
2016,ROBLOX,FAMILY,4.5,4449910,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up
2088,ROBLOX,FAMILY,4.5,4450855,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up
2206,ROBLOX,FAMILY,4.5,4450890,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up
4527,ROBLOX,FAMILY,4.5,4443407,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up


In [5]:
data = data.drop_duplicates(subset=['App'], keep='first')

## Category
**Since this is one entry I can easily find the correct info and plug it in.**
1. Category = Lifestyle
2. Rating = 1.9
3. Reviews = 19.0
4. Size = 3.0M
5. Installs = 1,000
6. Type = Free
7. Price = 0
8. Content Rating = Everyone

In [6]:
# the row with misinformation
data[data['Category'] == '1.9']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [7]:
# plugging in the correct information
data.at[10472,"Category"]="LIFESTYLE"
data.at[10472, "Rating"] = 1.9
data.at[10472, "Reviews"] = "19.0"
data.at[10472, "Size"] = "3.0M"
data.at[10472, "Installs"] = "1,000+"
data.at[10472, "Type"] = "Free"
data.at[10472,'Price']= 0
data.at[10472, 'Genres'] = 'Lifestyle'
data.at[10472, "Content Rating"] = "Everyone"
data.at[10472, "Last Updated"] = "February 11, 2018"

## Last Updated

In [8]:
#turn the dates into datetime and convert the date into days
data['Last Updated'] = pd.to_datetime(data['Last Updated'], format="%B %d, %Y", errors='coerce')
data['lastupdate'] = (data['Last Updated'] -  data['Last Updated'].max()).dt.days
data = data.drop(['Last Updated'], axis =1)

## Add new data
**After cleaning the data from Kaggle that was created two years ago, I decided that I would like more data from Google Play Store that contained apps that are popular now. This data was pulled from Google Play Store using webscraping and Api calls.**

In [9]:
#received data from file in notebook
new_data = pd.read_csv('apps_data.csv')
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2839 entries, 0 to 2838
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             2839 non-null   object 
 1   Installs        2836 non-null   object 
 2   Rating          2833 non-null   float64
 3   Reviews         2833 non-null   float64
 4   Price           2836 non-null   float64
 5   Size            2839 non-null   object 
 6   Genres          2839 non-null   object 
 7   Category        2839 non-null   object 
 8   Content Rating  2839 non-null   object 
 9   Current Ver     2839 non-null   object 
 10  Android Ver     2836 non-null   object 
 11  Type            2839 non-null   object 
 12  lastupdate      2839 non-null   int64  
dtypes: float64(3), int64(1), object(9)
memory usage: 288.5+ KB


In [10]:
#check for any missing information
new_data.isnull().sum()

App               0
Installs          3
Rating            6
Reviews           6
Price             3
Size              0
Genres            0
Category          0
Content Rating    0
Current Ver       0
Android Ver       3
Type              0
lastupdate        0
dtype: int64

In [11]:
#create a copy of the original
app_data = new_data.copy()
app_data

,App,Installs,Rating,Reviews,Price,Size,Genres,Category,Content Rating,Current Ver,Android Ver,Type,lastupdate
0,Geometry Dash,"1,000,000+",4.6,351474.0,1.99,85M,Arcade,GAME_ARCADE,Everyone,2.111,4.0 and up,Paid,-1248
1,Monopoly - Board game classic about real-estate!,"1,000,000+",4.3,25479.0,3.99,52M,Board,GAME_BOARD,Everyone,1.4.8,5.0 and up,Paid,-22
2,Stardew Valley,"1,000,000+",4.7,37001.0,4.99,80M,Role Playing,GAME_ROLE_PLAYING,Everyone 10+,1.4.5.151,4.4 and up,Paid,-94
3,Terraria,"1,000,000+",4.6,165091.0,4.99,161M,Adventure,GAME_ADVENTURE,Teen,1.4.0.5.2.1,4.4 and up,Paid,-129
4,Ultimate Custom Night,"100,000+",4.6,7796.0,2.99,151M,Action,GAME_ACTION,Teen,1.0.3,6.0 and up,Paid,-335
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2834,Archos Video Player Free,"5,000,000+",3.8,668.0,0.00,Varies with device,Video Players & Editors,VIDEO_PLAYERS,Everyone,10.2-20180416.1736,Varies with device,Free,-1093
2835,Compagnon t411,"100,000+",4.2,1218.0,0.00,3.5M,Social,SOCIAL,Everyone,2017.05-186,4.0 and up,Free,-1423
2836,Flud (Ad free),"50,000+",4.7,3452.0,1.49,Varies with device,Video Players & Editors,VIDEO_PLAYERS,Everyone,Varies with device,Varies with device,Paid,-423
2837,ezPDF Reader PDF Annotate Form,"500,000+",3.3,13296.0,3.99,33M,Productivity,PRODUCTIVITY,Everyone,2.7.1.0,4.0 and up,Paid,-336


In [12]:
# app_data = app_data.dropna()
# app_data

## Combine data

In [13]:
#combine new data with kaggle data
df = pd.concat([data, app_data])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12499 entries, 0 to 2838
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             12499 non-null  object 
 1   Category        12499 non-null  object 
 2   Rating          11030 non-null  float64
 3   Reviews         12493 non-null  object 
 4   Size            12499 non-null  object 
 5   Installs        12496 non-null  object 
 6   Type            12498 non-null  object 
 7   Price           12496 non-null  object 
 8   Content Rating  12499 non-null  object 
 9   Genres          12499 non-null  object 
 10  Current Ver     12491 non-null  object 
 11  Android Ver     12493 non-null  object 
 12  lastupdate      12499 non-null  int64  
dtypes: float64(1), int64(1), object(11)
memory usage: 1.3+ MB


In [14]:
#check for duplicates again
df['App'].value_counts(normalize=True)

Solitaire                                             0.00096
Udemy - Online Courses                                0.00032
DRAGON BALL Z DOKKAN BATTLE                           0.00032
Google Translate                                      0.00024
LEGO ® Batman: Beyond Gotham                          0.00024
                                                       ...   
25WPM Amateur ham radio Koch CW Morse code trainer    0.00008
Microsoft Authenticator                               0.00008
Allsetra B.V.                                         0.00008
Pint - FN Theme                                       0.00008
Forgotten Hill: Fall                                  0.00008
Name: App, Length: 11825, dtype: float64

In [15]:
df = df.drop_duplicates(subset=['App'], keep='first')

In [16]:
df= df.drop(columns={'Current Ver', 'Android Ver'}, axis=1)

In [17]:
df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,lastupdate
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,-213
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,-205
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,-7
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,-61
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,-49
...,...,...,...,...,...,...,...,...,...,...,...
2833,aTorrent PRO - torrent client,TOOLS,3.3,2385,Varies with device,"10,000+",Paid,1.99,Everyone,Tools,-1775
2834,Archos Video Player Free,VIDEO_PLAYERS,3.8,668,Varies with device,"5,000,000+",Free,0,Everyone,Video Players & Editors,-1093
2835,Compagnon t411,SOCIAL,4.2,1218,3.5M,"100,000+",Free,0,Everyone,Social,-1423
2837,ezPDF Reader PDF Annotate Form,PRODUCTIVITY,3.3,13296,33M,"500,000+",Paid,3.99,Everyone,Productivity,-336


## Rating

In [18]:
# replacing the NAN values with the average rating
df['Rating'] = df['Rating'].fillna(df['Rating'].mean())

## Reviews

In [19]:
df['Reviews'] = pd.to_numeric(df['Reviews'])

In [20]:
# df = df.dropna().reset_index(drop =True)
df['Reviews'] = df['Reviews'].fillna(df['Reviews'].mean())

In [21]:
df.Reviews.value_counts(normalize=True)

0.0          0.052600
1.0          0.023087
2.0          0.018605
3.0          0.015053
4.0          0.011924
               ...   
382100.0     0.000085
23802.0      0.000085
105766.0     0.000085
16876.0      0.000085
1163232.0    0.000085
Name: Reviews, Length: 6960, dtype: float64

In [22]:
review_mean = df.Reviews.mean()
df.Reviews = df['Reviews'].map(lambda x: review_mean if x == 0.0 else x)

## Size

In [23]:
df[df['Size'] == "Varies with device"]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,lastupdate
37,Floor Plan Creator,ART_AND_DESIGN,4.1,36639.0,Varies with device,"5,000,000+",Free,0,Everyone,Art & Design,-25
42,Textgram - write on photos,ART_AND_DESIGN,4.4,295221.0,Varies with device,"10,000,000+",Free,0,Everyone,Art & Design,-9
52,Used Cars and Trucks for Sale,AUTO_AND_VEHICLES,4.6,17057.0,Varies with device,"1,000,000+",Free,0,Everyone,Auto & Vehicles,-9
67,Ulysse Speedometer,AUTO_AND_VEHICLES,4.3,40211.0,Varies with device,"5,000,000+",Free,0,Everyone,Auto & Vehicles,-9
68,REPUVE,AUTO_AND_VEHICLES,3.9,356.0,Varies with device,"100,000+",Free,0,Everyone,Auto & Vehicles,-75
...,...,...,...,...,...,...,...,...,...,...,...
2813,Flud - Torrent Downloader,VIDEO_PLAYERS,4.5,122381.0,Varies with device,"10,000,000+",Free,0,Everyone,Video Players & Editors,-421
2817,Torrent Downloader,VIDEO_PLAYERS,3.8,3142.0,Varies with device,"1,000,000+",Free,0,Everyone,Video Players & Editors,-403
2824,BitTorrent® Pro - Official Torrent Download App,VIDEO_PLAYERS,4.4,3627.0,Varies with device,"50,000+",Paid,2.99,Everyone,Video Players & Editors,-14
2833,aTorrent PRO - torrent client,TOOLS,3.3,2385.0,Varies with device,"10,000+",Paid,1.99,Everyone,Tools,-1775


In [24]:
# create a variable to hold a certain value
mask = df['Size'].str.startswith('V')

#replace that value with zero
df.loc[mask, 'Size'] = 0

In [25]:
# convert the K and M to thousands and millions using regex
df.Size = (df.Size.replace(r'[kMG]+$', '', regex=True).astype(float) * \
           df.Size.str.extract(r'[\d\.]+([kMG]+)', expand=False).fillna(1)
           .replace(['k','M', 'G'], [2**10, 2**20, 2**30]).astype(int))

In [26]:
#convert column in megabytes
df['Size'] = (df['Size'] / 2**20).round(3)

In [27]:
#to retain some data replace zeros with the column's avearge
mean_size = df['Size'].mean()
df['Size'] = df.Size.mask(df.Size == 0.0,mean_size)

## Installs

In [28]:
# remove the + sign from columns
df['Installs'] = df['Installs'].str.replace('+','').str.replace(',','')

In [29]:
df['Installs'].unique()

array(['10000', '500000', '5000000', '50000000', '100000', '50000',
       '1000000', '10000000', '5000', '100000000', '1000000000', '1000',
       '500000000', '50', '100', '500', '10', '1', '5', '0', nan],
      dtype=object)

In [30]:
#change datatype
df['Installs'] = pd.to_numeric(df['Installs'])
df['Installs'] = df['Installs'].replace(np.nan, 0)

In [31]:
install_mean = df['Installs'].mean()
df['Installs'] = df['Installs'].map(lambda x: install_mean if x == 0.0 else x)

## Free Apps

In [32]:
df.Type.value_counts(normalize=True)

Free    0.875592
Paid    0.124408
Name: Type, dtype: float64

In [33]:
df['is_free'] = df['Type'].map(lambda x: 1 if x == 'Free' else 0)

In [34]:
df['Price'] = df['Price'].apply(lambda x : str(x).strip('$')).astype(float).round()

In [35]:
# from sklearn import preprocessing
# def labelencode(col, df):
#     le = preprocessing.LabelEncoder()
#     df[col] = le.fit_transform(df[col])
# le = preprocessing.LabelEncoder()
# df['Content Rating'] = le.fit_transform(df['Content Rating'])

# le = preprocessing.LabelEncoder()
# df['Genres'] = le.fit_transform(df['Genres'])

In [38]:
main_df = df.copy()
main_df = main_df.drop(['App', 'Type'], axis=1)
main_df.dropna(inplace=True)

# Cleaned Data

In [39]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11822 entries, 0 to 2838
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Category        11822 non-null  object 
 1   Rating          11822 non-null  float64
 2   Reviews         11822 non-null  float64
 3   Size            11822 non-null  float64
 4   Installs        11822 non-null  float64
 5   Price           11822 non-null  float64
 6   Content Rating  11822 non-null  object 
 7   Genres          11822 non-null  object 
 8   lastupdate      11822 non-null  int64  
 9   is_free         11822 non-null  int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 1016.0+ KB


In [40]:
main_df.to_csv('final_data1.csv')